In [472]:
import spacy
from spacy import displacy 
import os
import nltk
from nltk.tag import StanfordNERTagger
from allennlp.predictors.predictor import Predictor

nlp = spacy.load('en_core_web_sm',disable=['ner','textcat'])


java_path = "Java/jre1.8.0_251/bin/java.exe"
os.environ['JAVAHOME'] = java_path
st = StanfordNERTagger(r'SentiSE-master/edu/stanford/nlp/models/ner/english.all.3class.caseless.distsim.crf.ser.gz',
                           r'stanford-ner-2020-11-17/stanford-ner.jar',
                           encoding='utf-8')


predictor = Predictor.from_path("I17-1099.Datasets/EMNLP_dataset/coref-spanbert-large-2021.03.10(3).tar.gz")


I0731 19:59:59.763079 28844 plugins.py:96] Plugin allennlp_models available
I0731 19:59:59.788581 28844 archival.py:211] loading archive file C:/Users/Windows dunya/Desktop/fourth year/5th/I17-1099.Datasets/EMNLP_dataset/coref-spanbert-large-2021.03.10(3).tar.gz from cache at C:\Users\Windows dunya\Desktop\fourth year\5th\I17-1099.Datasets\EMNLP_dataset\coref-spanbert-large-2021.03.10(3).tar.gz
I0731 19:59:59.795759 28844 archival.py:300] extracting archive file C:\Users\Windows dunya\Desktop\fourth year\5th\I17-1099.Datasets\EMNLP_dataset\coref-spanbert-large-2021.03.10(3).tar.gz to temp dir C:\Users\WINDOW~1\AppData\Local\Temp\tmpykki_15k
W0731 20:00:12.973821 28844 params.py:20] error loading _jsonnet (this is expected on Windows), treating C:\Users\WINDOW~1\AppData\Local\Temp\tmpykki_15k\config.json as plain json
I0731 20:00:12.984021 28844 params.py:221] dataset_reader.type = coref
I0731 20:00:12.985994 28844 params.py:221] dataset_reader.max_instances = None
I0731 20:00:12.986993

In [508]:
SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
OBJECTS = ["dobj", "dative", "attr", "oprd"]
ADJECTIVES = ["acomp", "advcl", "advmod", "amod", "appos", "nn", "nmod", "ccomp", "complm","hmod", "infmod", "xcomp", "rcmod", "poss"," possessive"]
COMPOUNDS = ["compound"]
PREPOSITIONS = ["prep"]


doc="i heard she has passed the bar exam and married recently "

doc = nlp(doc)
displacy.render(doc, style='dep',jupyter=True)


#accept(nsubjectpass) finish approved completed

In [509]:
doc=nlp(doc)
for tok in doc: 
    head=tok.head
    print(tok.text, "-->",tok.dep_,"-->", tok.pos_,"-->",tok.lemma_," ...", tok.head,",,,",tok.n_lefts) 
    if tok.pos_=="VERB":
        for i in tok.rights:
            if str(i.text)=="divorce" and i.dep_=="dobj":
                print(".........................",i)
            

i --> nsubj --> PRON --> I  ... heard ,,, 0
heard --> ROOT --> VERB --> hear  ... heard ,,, 1
she --> nsubj --> PRON --> she  ... passed ,,, 0
has --> aux --> AUX --> have  ... passed ,,, 0
passed --> ccomp --> VERB --> pass  ... heard ,,, 2
the --> det --> DET --> the  ... exam ,,, 0
bar --> compound --> NOUN --> bar  ... exam ,,, 0
exam --> dobj --> NOUN --> exam  ... passed ,,, 2
and --> cc --> CCONJ --> and  ... passed ,,, 0
married --> conj --> VERB --> marry  ... passed ,,, 0
recently --> advmod --> ADV --> recently  ... married ,,, 0


# intializing lists of  events keywords

In [476]:
noun_direct_relation=["wedding","anniversary","graduation","birthday","interview","meeting","resignation","job"]
nouns=["pain","birth","surgery","job","cold","flu","fever","wedding","meeting","interview","offer","promotion","headache","exam","divorce","contract","relationship","diploma","masters"]#as object to the aux verb
verbs_without_need_objects=["wed","divorce","divorced","elect","engage","die","married","nominate","promote","marry","graduate","bear","hurt","meet","interview"]
verb_with_prepo=["move","go","travel"]
adjectives=["sick","ill","pregnant"]
pronoun=["my","his","her","their","your"]
prepo=["to","into","in","on","for"]
verbs_with_objects={"celebrate":["birthday","birth","graduation","anniversary","success","news"],
                   "graduate":["collage","school","university"],
                   "win":["support","award","honor","scholarship","prize","lawsuit","pounds","weight"],
                   "lose":["support","award","honor","scholarship","prize","lawsuit","pounds","weight"],
                   "admit":["university","collage","offer","school"],
                   "pass":["exam","test","semester","midterms","school","interview","meeting"],
                   "present":["essay","thesis","reading","statment","presentation","dissertation","project","research","paper"],
                   "discuss":["essay","thesis","reading","statment","presentation","dissertation","project","research","paper"],
                   "defend":["essay","thesis","dissertation","project","research","paper"],
                   "move":["city","home","apartment","town"],
                   "travel":["city","home","apartment","town"],
                   "contract":["agreement","meeting"],
                   "act":["role","series","movie","theater"],
                   "publish":["book","post","cover","copy"],
                   "pregnant":["baby","boy","girl"],
                   "buy":["car","house","phone","laptope"],
                   "accept":["invite","work","school","business ","university","job","offer","college","program","project","research","paper","invitation"],
                   "visit":["hospital","doctor","city","country"],
                   "go":["trip","vacation","holiday"],
                   "sing":["song","album"],
                   "sold":["car","house","phone"],
                   "sign":["deal","contruct"],
                   "finish":["diploma","masters","trip","vacation","holiday","college","university","job","test","exam","school","semester","midterms","album","book","essay","thesis","reading","statment","presentation","dissertation","project","research","paper"],
                   "start":["diploma","masters","trip","vacation","holiday","college","university","job","test","exam","school","semester","midterms","album","book","essay","thesis","reading","statment","presentation","dissertation","project","research","paper"],
                   "break":["leg","arm","finger","neck","head","Back"],
                   "mark":["anniversary","birthday","birth","graduation","success","death"],
                   "fail":["test","exam","school","semester","midterms"]}


# rules for extraction events patterns

In [477]:
class Rules:
    def __init__(self,text,list_of_keywords=[],tokens=[],tokens_=[],pos_tags=[],dependency=[],lemma=[]):
        self.text=text
        self.list_of_keywords=list_of_keywords
        self.tokens=[]
        self.tokens_=[]
        self.pos_tags=[]
        self.dependency=[]
        self.lemma=[]
        for tok in self.text: 
            self.tokens.append(tok.text)
            self.tokens_.append(tok)
            self.pos_tags.append(tok.pos_)
            self.dependency.append(tok.dep_)
            self.lemma.append(tok.lemma_)
        

    def retrival_senetce_rule1(self):
        for i,lemm in enumerate(self.lemma):
            
                if lemm in self.list_of_keywords and (self.pos_tags[i]=='VERB' ):
                    #or self.pos_tags[i]=='ADV' or self.pos_tags[i]=='ADJ' or self.pos_tags[i]=='NOUN'
                    return self.text,self.tokens[i]
             

            
    def retrival_senetce_rule2(self):
        for i,lemm in enumerate(self.lemma):
            if lemm in self.list_of_keywords and self.pos_tags[i]=='VERB':
                for t in self.tokens_[i].rights:
                    if t.dep_=="dobj" and str(t.lemma_) in self.list_of_keywords[lemm]:
                        return self.text,self.tokens[i]+" "+t.text
                    
                    elif t.dep_=="dobj" :
                        for t1 in t.rights:
                            if t1.dep_=="prep" :
                                for t2 in t1.rights:
                                    if t2.dep_=="pobj" and str(t2.lemma_) in self.list_of_keywords[lemm]:
                                        return self.text,self.tokens[i]+" "+t2.text

                    elif t.dep_=="oprd" and str(t.lemma_) in self.list_of_keywords[lemm] :
                        return self.text,lemmself.tokens[i]+" "+t.text

                    elif t.dep_=="prep":
                        for t1 in t.rights:
                            if t1.dep_=="pobj" and str(t1.lemma_) in self.list_of_keywords[lemm]:
                                return self.text,self.tokens[i]+" "+t1.text
                            
                            elif t1.dep_=="pobj":
                                for t4 in t1.lefts:
                                    if t4.dep_=="compound" and str(t4.lemma_) in self.list_of_keywords[lemm]:
                                        return self.text,self.tokens[i]+" "+t4.text
                 
                                    
                for t in self.tokens_[i].lefts:
                    if t.dep_=="nsubjpass" and str(t) in self.list_of_keywords[lemm]:
                        return self.text,self.tokens[i]+" "+t.text
                     
            
            
    def retrival_senetce_rule3(self):  
        for i,lemm in enumerate(self.lemma):
            if self.pos_tags[i]=='VERB'or self.pos_tags[i]== 'AUX':
                for t in self.tokens_[i].rights:
                    if t.dep_=="dobj" and str(t) in self.list_of_keywords:
                        return self.text,self.tokens[i]+" "+t.text
                    
                    elif t.dep_=="prep":
                        for t1 in t.rights:
                            if t1.dep_=="pobj" and str(t1) in self.list_of_keywords:
                                return self.text,self.tokens[i]+" "+t1.text
                    
                    elif t.dep_=="dobj" :
                        for t1 in t.rights:
                            if t1.dep_=="prep" :
                                for t2 in t1.rights:
                                    if t2.dep_=="pobj" and str(t2) in self.list_of_keywords:
                                        return self.text,self.tokens[i]+" "+t2.text
                                    
                     
            
            
        
    def retrival_senetce_rule4(self):
        for i,lemm in enumerate(self.lemma):
            if self.pos_tags[i]=="AUX":
                for t in self.tokens_[i].rights:
                    if t.dep_=="acomp" and str(t) in self.list_of_keywords and t.pos_=="ADJ":
                        print("ad ", t.text)
                        return self.text,self.tokens[i]+" "+t.text
                    
                
            elif self.pos_tags[i]=="VERB":
                for t in self.tokens_[i].rights:
                     if t.dep_=="oprd" and str(t) in self.list_of_keywords and (t.pos_=="NOUN" or t.pos_=="ADJ"):
                        return self.text,self.tokens[i]+" "+t.text
                     
            
    
    
    def retrival_senetce_rule5(self): 
        for i,lemm in enumerate(self.lemma):
            if (self.pos_tags[i]=="PRON" or self.pos_tags[i]=="PROPN" ) :
                head=self.tokens_[i].head
                if self.tokens_[i].dep_=="poss" and str(head) in self.list_of_keywords and head.pos_=="NOUN":
                        return self.text,self.tokens[i]+" "+head.text
                    
                elif self.tokens_[i].dep_=="poss" and head.pos_=="NOUN":
                    for t in head.lefts:
                        if str(t) in self.list_of_keywords and  t.dep_=="compound":
                            return self.text,self.tokens[i]+" "+t.text
                        
                
                            
    def retrival_senetce_rule6(self):
        for i,lemm in enumerate(self.lemma):
            if lemm in self.list_of_keywords and self.pos_tags[i]=='VERB':
                for t in self.tokens_[i].rights:
                    if t.dep_=="prep" and str(t) in prepo:
                        return self.text,self.tokens[i]+" "+t.text
                """for t in self.tokens_[i].lefts:
                    if t.dep_=="aux" and str(t) == "to":
                        return None"""
                    
                    
    def retrival_senetce_rule7(self):
        for i,lemm in enumerate(self.lemma):
            if lemm in self.list_of_keywords and (self.pos_tags[i]=='VERB' or self.pos_tags[i]=='NOUN' or self.pos_tags[i]=='ADJ'):
                    if self.tokens_[i].dep_=="ROOT" :
                        for t in self.tokens_[i].rights:
                            if t.dep_=="dobj" and str(t.lemma_) in self.list_of_keywords[lemm]:
                                return self.text

                            elif t.dep_=="dobj" :
                                for t1 in t.rights:
                                    if t1.dep_=="prep" :
                                        for t2 in t1.rights:
                                            if t2.dep_=="pobj" and str(t2.lemma_) in self.list_of_keywords[lemm]:
                                                return self.text

                            elif t.dep_=="oprd" and str(t.lemma_) in self.list_of_keywords[lemm] :
                                return self.text

                            elif t.dep_=="prep":
                                for t1 in t.rights:
                                    if t1.dep_=="pobj" and str(t1.lemma_) in self.list_of_keywords[lemm]:
                                        return self.text
                                    elif t1.dep_=="pobj":
                                        for t4 in t1.lefts:
                                            if t4.dep_=="compound" and str(t4.lemma_) in self.list_of_keywords[lemm]:
                                                return self.text
        
    """
                            
    def retrival_senetce_rule10(self):
        for i,lemm in enumerate(self.lemma):
            if (self.pos_tags[i]=='VERB' or self.pos_tags[i]=="AUX"):
                    for t in self.tokens_[i].rights:
                        if t.dep_=="acomp" and str(t) in self.list_of_keywords and (t.pos_=="ADJ" or t.pos_=="VERB" or t.pos_=="ADV"): 
                            return self.text
                        
    def retrival_senetce_rule11(self):
        for i,lemm in enumerate(self.lemma):
            if ( self.pos_tags[i]=="AUX" and self.dependency[i]=="auxpass"):
                        head=self.tokens_[i].head
                        if  str(head) in self.list_of_keywords and (head.pos_=="ADJ" or head.pos_=="VERB"): 
                            return self.text
                            
                            """
    



In [478]:
doc="you and me will married this month ."
doc=nlp(doc)
p1=Rules(doc, verbs_without_need_objects)
text=p1.retrival_senetce_rule1()
text

(you and me will married this month ., 'married')

In [479]:
doc="he passed the bar exam"
doc=nlp(doc)
p2=Rules(doc, verbs_with_objects)
text=p2.retrival_senetce_rule2()
text

(he passed the bar exam, 'passed exam')

In [480]:
doc="i got a surgery"
doc=nlp(doc)
p3=Rules(doc, nouns)
text=p3.retrival_senetce_rule3()
text

(i got a surgery, 'got surgery')

In [481]:
doc="i am very sick"
doc=nlp(doc)
p4=Rules(doc, adjectives)
text=p4.retrival_senetce_rule4()
text

ad  sick


(i am very sick, 'am sick')

In [482]:
doc="my wedding was last year"
doc=nlp(doc)
p5=Rules(doc, noun_direct_relation)
text=p5.retrival_senetce_rule5()
text

(my wedding was last year, 'my wedding')

In [483]:
doc="i am moving to USA"
doc=nlp(doc)
p6=Rules(doc, verb_with_prepo)
text=p6.retrival_senetce_rule6()
text

(i am moving to USA, 'moving to')

# checking persons subjects

In [484]:
p=["i","we","you","they","he","she","my","his","her","their","your","our","me","him","them"]
def is_person(text,subject,st=st):
    x=0
    tokens=[]
    for token in text :
        tokens.append(token)
    for i,token in enumerate(tokens):
        if str(token) == str(subject):  
            lis=str(text).split()
            for i,j in enumerate(lis):
                j=j.replace("'s","")
                lis[i]=j
            #print(lis)
            
            #print(tagged)
            if token.pos_=="NOUN":
                tagged = st.tag(lis)
                for i,j in enumerate(tagged):
                    if 'PERSON' in j:
                        if str(j[0])==str(subject):
                            return True 
                        if str(j[0])!=str(subject):
                            return False
                for i,j in enumerate(tagged):
                    if 'PERSON' not in j:
                        x+=1
                        if x==len(tokens):
                            return False
                    
            elif token.pos_=="PRON" and (str(token) in p):
                return True
            elif token.pos_=="PRON" and (str(token) not in p):
                return False
            
            elif token.pos_=="PROPN":
                return True
            


In [485]:
text="george divorced katty and take her alone"
text=nlp(text)
is_person(text,"george",st=st)

True

# rules for subjects extraction

In [501]:
obj_pronoun=["me","him","her","them","you","us"]
p_pronoun=["my","his","her","their","your","our"]
from collections import Counter
import pandas as pd

def find_obj_for_verb(doc,verb):
    tokens=[]
    for token in doc :
         tokens.append(token)
    for i,token in enumerate(tokens):
            if str(token) == str(verb):
                for t in tokens[i].rights:
                    if t.dep_=="dobj" :
                        return t

                    elif t.dep_=="prep":
                        for t1 in t.rights:
                            if t1.dep_=="pobj" :
                                return t1
                                    
                for t in tokens[i].lefts:
                    if t.dep_=="nsubjpass" :
                        if token.n_rights>0:
                            for t1 in token.rights:
                                if t1.dep_ == "agent":
                                    return t
                                    
    
    
def getSubsFromConjunctions(subs):
    moreSubs = []
    rightDeps=[]
    for sub in subs:
        for tok in sub.rights:
            if tok.dep_ =="conj" :
                    #.................................................................
                    subs=chek_pronoun_verb(tok)
                    moreSubs.append(subs)
            if len(moreSubs) > 0:
                
                moreSubs.extend(getSubsFromConjunctions(moreSubs))
    return moreSubs
            
        

def chek_pronoun_verb(tok):
    if  tok.n_lefts>0:
        for s in (tok.lefts):
            if  (s.dep_=="poss") and str(s) in p_pronoun :
                return s
            elif  (s.dep_=="poss") and (s.pos_=="PROPN" or s.pos_=="NOUN"):
                return s
            else:
                return tok
    else :
        return tok

def chek_left_sub(tok):
    if  tok.n_lefts>0:
        for s in tok.lefts:
            if  (s.dep_=="compound") and (s.pos_=="NOUN" or s.pos_=="PROPN") :
                return s
            else :
                return tok
    else :
        return tok

def find_sub_for_verb(doc,verb):
    tokens=[]
    subs=[]
    objs=[]
    s=[]
    e=[]
    p=[]
    for token in doc :
         tokens.append(token)
    for i,token in enumerate(tokens):
            if str(token) == str(verb):
                for tok in token.lefts:                    
                    if tok.dep_ == "auxpass" :
                        if tok.pos_=='AUX' :
                            for t1 in tok.lefts:
                                if t1.dep_=="nsubj" or t1.dep_=="nsubjpass" :
                                    subs.append(t1)
                                            
                                    
                    elif tok.dep_ == "nsubjpass":
                        objs.append(tok)
                        if token.n_rights>0:
                            for t in token.rights:
                                if t.dep_ == "agent":
                                    for t1 in t.rights:
                                        if t1.dep_=="pobj":
                                            subs.append(t1)
                                else:
                                    subs.append(tok)

                        else:
                            subs.append(tok)
                           
                        
                    elif tok.dep_ == "nsubj" :
                        if is_person(doc,tok)==True:
                                subs.append(tok)
                        
                        else :
                            for t3 in tok.lefts:
                                if t3.dep_=="poss" and (t3.pos_=="NOUN" or t3.pos_=="PROPN" or t3.pos_=="PRON"):
                                    subs.append(tok)
                                else:    
                                    for t2 in token.rights:
                                        if t2.dep_ == "nsubj" :
                                            #if is_person(doc,t2)==True:
                                            subs.append(t2)
                                    
                                
                                
                        
            #kareem get to finish his masters 
            if len(subs)==0:
                if token.dep_ =="acomp" or token.dep_ =="xcomp":
                        head=token.head
                        if head.pos_=="VERB" or head.pos_=="AUX":
                            for t in head.lefts:
                                if t.dep_=="nsubj" or t.dep_=="nsubjpass":
                                    subs.append(t)
                                    
                elif token.dep_=="conj":
                    #print("conj 1")
                    h=token.head
                    if h.pos_=="VERB":
                        sub=find_sub_for_verb(doc,h)[0]
                        if sub :
                            subs.append(sub) 
                        else:
                            subs.append(None)
                        
                
                                
                
                    
                
            if len(subs) > 0:
                subs = list(pd.Series(subs).drop_duplicates())
                subs.extend(getSubsFromConjunctions(subs))
                for i in subs:
                    s.append(chek_pronoun_verb(i))
               
                for i in s:
                    if is_person(doc,i)==False :
                        s.remove(i)
                
                for i in range(len(s)):
                    if s[i]!=subs[i] and str(s[i]) in p_pronoun:
                        p.append((s[i],subs[i]))
                    elif s[i]!=subs[i] and (s[i].pos_=="PROPN" or s[i].pos_=="NOUN"):
                        p.append((s[i],subs[i]))
                        
                    elif s[i]==subs[i]:
                        
                        p.append(s[i])
                    else:
                        p.append(s[i])
                        p.append(subs)
                        
                        
                        
                return p


In [503]:
    
def findSubs_for_verbEvents(doc,list_=[]):
    
    subs=[]
    objs=[]
    for token in doc:
        if token.pos_ == "VERB" and str(token.lemma_) in list_:
            obj=find_obj_for_verb(doc,token)
            sub=find_sub_for_verb(doc,token)
            if sub:
                return sub
                
            else:
                if obj!=None:
                    if str(obj) in obj_pronoun:
                        return [obj]



def findSubs_for_direct_relation(doc,list_=[]):
    tokens=[]
    subs=[]
    for token in doc :
         tokens.append(token)
    for i,token in enumerate(tokens):
        if (tokens[i].pos_=="PRON" or tokens[i].pos_=="PROPN" ) :
            head=tokens[i].head
            if tokens[i].dep_=="poss" and str(head) in list_ and head.pos_=="NOUN":
                subs.append(token) 
                    
            elif tokens[i].dep_=="poss" and head.pos_=="NOUN":
                for t in head.lefts:
                    if str(t) in list_ and  t.dep_=="compound":
                        subs.append(token) 
        if len(subs) > 0:
                return subs 
            
        
def findSubs_for_nounsEvents(doc,list_=[]):
    tokens=[]
    subs=[]
    for token in doc :
         tokens.append(token)
    for i,token in enumerate(tokens):
        if str(token) in list_:
            
            
            s=chek_pronoun_verb(token)
            if str(s) == str(token):
                
                for i,token in enumerate(tokens):
                    if tokens[i].pos_=='VERB'or tokens[i].pos_== 'AUX':
                        obj=find_obj_for_verb(doc,token)
                        sub=find_sub_for_verb(doc,token)
                        for t in tokens[i].rights:
                            if t.dep_=="dobj" and str(t) in list_:
                                if sub :
                                    if len(sub)>0:
                                        return sub 


                            elif t.dep_=="prep":
                                for t1 in t.rights:
                                    if t1.dep_=="pobj" and str(t1) in list_:
                                        if sub:
                                            if len(sub)>0:
                                                return sub 

                            elif t.dep_=="dobj" :
                                for t1 in t.rights:
                                    if t1.dep_=="prep" :
                                        for t2 in t1.rights:
                                            if t2.dep_=="pobj" and str(t2) in list_:
                                                if sub:
                                                    if len(sub)>0:
                                                        return sub 
            
            elif s !=None and str(s) != str(token) :
                return [s]

                       
                                    

def findSubs_for_adjectiveEvents(doc,list_=[]):
    tokens=[]
    subs=[]
    for token in doc :
         tokens.append(token)
    for i,token in enumerate(tokens):
            if tokens[i].pos_=="AUX":
                obj=find_obj_for_verb(doc,token)
                sub=find_sub_for_verb(doc,token)
                for t in tokens[i].rights:
                    if t.dep_=="acomp" and str(t) in list_ and t.pos_=="ADJ":
                        if sub:
                            if len(sub )>0:
                                return sub
                
            elif tokens[i].pos_=="VERB":
                obj=find_obj_for_verb(doc,token)
                sub=find_sub_for_verb(doc,token)
                for t in tokens[i].rights:
                     if t.dep_=="oprd" and str(t) in list_ and (t.pos_=="NOUN" or t.pos_=="ADJ"):
                        if sub :
                            if len(sub)>0:
                                return sub
     
                            
def findSubs_for_actionEvents(doc,list_=[]):
    tokens=[]
    subs=[]
    for token in doc :
        tokens.append(token)
        for i,token in enumerate(tokens):
            if str(tokens[i].lemma_) in list_ and tokens[i].pos_=='VERB':
                obj=find_obj_for_verb(doc,token)
                sub=find_sub_for_verb(doc,token)
                for t in tokens[i].rights:
                    if t.dep_=="prep" :
                        if sub:
                            if len(sub)>0:
                                return sub                              
                

# checking events extraction and subjects extraction rules 

In [402]:
f = open("Dialogs_data.txt", "r")
sentence=[]
sayers=[]
for line in f:

        stripped_line = line.strip()
        
        sayer=stripped_line[:stripped_line.find(":")]
        sayers.append(sayer)
        
        stripped_line=stripped_line[stripped_line.find(":")+1:]
        sentence.append(stripped_line)

f.close()
n=0

for i in sentence :
    print(i)
    list_tuple=[]
    t=i.lower()
    doc=nlp(t)

    
    p1=Rules(doc, verbs_without_need_objects)
    text=p1.retrival_senetce_rule1()
    if text!=None:
        list_tuple.append(["Rule1 ",text , findSubs_for_verbEvents(nlp(text[0]),verbs_without_need_objects)])

        
    p2=Rules(doc, verbs_with_objects)
    text=p2.retrival_senetce_rule2()
    if text!=None:
        list_tuple.append(["Rule2 ",text ,findSubs_for_verbEvents(nlp(text[0]),verbs_with_objects)])
             
                
    p3=Rules(doc, nouns)
    text=p3.retrival_senetce_rule3()
    if text!=None:
        list_tuple.append(["Rule3 ",text ,findSubs_for_nounsEvents(nlp(text[0]),nouns)])
    
    
    p4=Rules(doc, adjectives)
    text=p4.retrival_senetce_rule4()
    if text!=None:
        list_tuple.append(["Rule4 ",text ,findSubs_for_adjectiveEvents(nlp(text[0]),adjectives)])
    
    
    p5=Rules(doc, noun_direct_relation)
    text=p5.retrival_senetce_rule5()
    if text!=None:
        list_tuple.append(["Rule5 ",text , findSubs_for_direct_relation(nlp(text[0]),noun_direct_relation)])
    
    
    p6=Rules(doc, verb_with_prepo)
    text=p6.retrival_senetce_rule6()
    if text!=None:
        list_tuple.append(["Rule6 ",text ,findSubs_for_actionEvents(nlp(text[0]),verb_with_prepo)])
    
    
    for i in list_tuple:
        print(".... ",i)
        
    if i=="":
        n+=1
        print("\n")
print(n)        

That's why we want to have a look at other hotels .
We donâ€™t want to spend too much on an extravagant wedding reception .
Iguess you're right .



george and my husband will married this week .
....  ['Rule1 ', (george and my husband will married this week ., 'married'), [george, my]]



george and mari will married this month .
....  ['Rule1 ', (george and mari will married this month ., 'married'), [george, mari]]
i wish a lot .



i and me will married this month .
....  ['Rule1 ', (i and me will married this month ., 'married'), [i, me]]
i wish a lot .



you and me will married this month .
....  ['Rule1 ', (you and me will married this month ., 'married'), [you, me]]
i wish a lot .



I'll stay home this morning and rest , but if I feel better in the afternoon , I'm going to the meeting .
....  ['Rule3 ', (i'll stay home this morning and rest , but if i feel better in the afternoon , i'm going to the meeting ., 'going meeting'), [i]]
....  ['Rule6 ', (i'll stay home this mornin

# filtering all cases for rules conflict and aggregiation all possible (events and subjects) for the sentence in one list

In [490]:
def equal_subjects(subject1,subject2):
    p_feminine=["she","her"]
    p_masculine=["he","his","him"]
    p_self_speaker=["i","my","me"]
    p_you=["you","your"]
    p_they=["they","their"]
    p_we=["we","our","us"]
    
    if str(subject1)==str(subject2):
        return True
    elif str(subject1) in p_feminine and str(subject2) in p_feminine:
        return True
    elif str(subject1) in p_masculine and str(subject2) in p_masculine:
        return True
    elif str(subject1) in p_self_speaker and str(subject2) in p_self_speaker:
        return True
    elif str(subject1) in p_you and str(subject2) in p_you:
        return True
    elif str(subject1) in p_they and str(subject2) in p_they:
        return True
    elif str(subject1) in p_we and str(subject2) in p_we:
        return True
    else:
        False


    
def aggregation_for_sentence(list_tuple):
    ag=[]
    events=[]
    subjects=[]
    sub=[]
    s=[]
    if len(list_tuple)>=1: 
        for i in range(len(list_tuple)):
            #event
            lis1=list_tuple[i][1]
            #subject
            lis2=list_tuple[i][2]
            if lis2 !=None :
                if lis1 !=None:
                    ag.append([lis1[1],lis2])
            
            
        for i in range(len(ag)):
            events.append(ag[i][0])
            
            #has subject
            if len(ag[i])==2:
                sub_=ag[i][1]
                for index in range(len(sub_)):
                    if isinstance(sub_[index], tuple)==False:
                        if sub_[index] not in s :
                            s.append(sub_[index])
                            if s!=None:
                                if len(s)!=0:
                                    for j in range(len(s)):
   
                                        subjects.append(s[j])
                                  
                                        
                    else:
                        subjects.append(sub_[index])
                        
        subjects = list(pd.Series(subjects).drop_duplicates())      
        return events,subjects
        
        

        
        
        

def filtering(list_tuple=[]):
    
    if len(list_tuple)>=6: 
        _6_rules(list_tuple)
        if len(list_tuple)>=5:
            _5_rules(list_tuple)
            if len(list_tuple)>=4:
                _4_rules(list_tuple)
                if len(list_tuple)>=3:
                    _3_rules(list_tuple)
                    if len(list_tuple)>=2:
                        _2_rules(list_tuple)
    
    
    if len(list_tuple)>=5: 
        _5_rules(list_tuple)
        if len(list_tuple)>=4:
            _4_rules(list_tuple)
            if len(list_tuple)>=3:
                _3_rules(list_tuple)
                if len(list_tuple)>=2:
                    _2_rules(list_tuple)
        
    if len(list_tuple)>=4:
        _4_rules(list_tuple)
        if len(list_tuple)>=3:
            _3_rules(list_tuple)
            if len(list_tuple)>=2:
                    _2_rules(list_tuple)
        
    if len(list_tuple)>=3:
        _3_rules(list_tuple)
        if len(list_tuple)>=2:
            _2_rules(list_tuple)
            
    if len(list_tuple)>=2:
        _2_rules(list_tuple)
       
                 
def _6_rules(list_tuple=[]):   
    
            if (list_tuple[0][0]=="Rule1 "  and list_tuple[1][0]=="Rule2 "  and list_tuple[2][0]=="Rule3 " and list_tuple[3][0]=="Rule4 "  and list_tuple[4][0]=="Rule5 "  and list_tuple[5][0]=="Rule6 ") :
                        lis1=str(list_tuple[2][1][1]).split()
                        lis2=str(list_tuple[5][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            print(lis1[0]+" "+lis2[1]+" "+lis1[1])
                            list_tuple.remove(list_tuple[2])
                            list_tuple.remove(list_tuple[5])
        
def _5_rules(list_tuple=[]):
    

            if (list_tuple[0][0]=="Rule1 "  and list_tuple[1][0]=="Rule2 "  and list_tuple[2][0]=="Rule3 " and list_tuple[3][0]=="Rule4 "  and list_tuple[4][0]=="Rule5 ") :
                        lis1=str(list_tuple[2][1][1]).split()
                        lis2=str(list_tuple[4][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            list_tuple.remove(list_tuple[4])


            elif (list_tuple[0][0]=="Rule1 "  and list_tuple[1][0]=="Rule2 " and list_tuple[2][0]=="Rule3 "  and list_tuple[3][0]=="Rule4 "  and list_tuple[4][0]=="Rule5 "):
                        lis1=str(list_tuple[1][1][1]).split()
                        lis2=str(list_tuple[4][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            list_tuple.remove(list_tuple[4])
                            
                            
            elif (list_tuple[0][0]=="Rule1 "  and list_tuple[1][0]=="Rule2 " and list_tuple[2][0]=="Rule3 "  and list_tuple[3][0]=="Rule4 "  and list_tuple[4][0]=="Rule6 "):
                        lis1=str(list_tuple[2][1][1]).split()
                        lis2=str(list_tuple[4][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            print(lis1[0]+" "+lis2[1]+" "+lis1[1])
                            list_tuple.remove(list_tuple[2])
                            list_tuple.remove(list_tuple[4])
                            
            elif (list_tuple[0][0]=="Rule1 "  and list_tuple[1][0]=="Rule2 " and list_tuple[2][0]=="Rule3 "  and list_tuple[3][0]=="Rule5 "  and list_tuple[4][0]=="Rule6 "):
                        lis1=str(list_tuple[2][1][1]).split()
                        lis2=str(list_tuple[4][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            print(lis1[0]+" "+lis2[1]+" "+lis1[1])
                            list_tuple.remove(list_tuple[2])
                            list_tuple.remove(list_tuple[4])
                            
            elif (list_tuple[0][0]=="Rule2 "  and list_tuple[1][0]=="Rule3 " and list_tuple[2][0]=="Rule4 "  and list_tuple[3][0]=="Rule5 "  and list_tuple[4][0]=="Rule6 "):
                        lis1=str(list_tuple[1][1][1]).split()
                        lis2=str(list_tuple[4][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            print(lis1[0]+" "+lis2[1]+" "+lis1[1])
                            list_tuple.remove(list_tuple[1])
                            list_tuple.remove(list_tuple[4])
                            
            elif (list_tuple[0][0]=="Rule1 "  and list_tuple[1][0]=="Rule3 " and list_tuple[2][0]=="Rule4 "  and list_tuple[3][0]=="Rule5 "  and list_tuple[4][0]=="Rule6 "):
                        lis1=str(list_tuple[1][1][1]).split()
                        lis2=str(list_tuple[4][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            print(lis1[0]+" "+lis2[1]+" "+lis1[1])
                            list_tuple.remove(list_tuple[1])
                            list_tuple.remove(list_tuple[4])
                            
                            
def _4_rules(list_tuple=[]):    
                                

            if (list_tuple[0][0]=="Rule1 "  and list_tuple[1][0]=="Rule2 "  and list_tuple[2][0]=="Rule3 "  and list_tuple[3][0]=="Rule5 "):
                        lis1=str(list_tuple[0][1][1]).split()
                        lis2=str(list_tuple[3][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            list_tuple.remove(list_tuple[3])

            elif (list_tuple[0][0]=="Rule2 "  and list_tuple[1][0]=="Rule3 "  and list_tuple[2][0]=="Rule4 "  and list_tuple[3][0]=="Rule5 "):
                        lis1=str(list_tuple[0][1][1]).split()
                        lis2=str(list_tuple[3][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            list_tuple.remove(list_tuple[3])

            elif (list_tuple[0][0]=="Rule1 "  and list_tuple[1][0]=="Rule2 "  and list_tuple[2][0]=="Rule4 "  and list_tuple[3][0]=="Rule5 "):
                        lis1=str(list_tuple[1][1][1]).split()
                        lis2=str(list_tuple[3][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            list_tuple.remove(list_tuple[3])

            elif (list_tuple[0][0]=="Rule1 "  and list_tuple[1][0]=="Rule3 "  and list_tuple[2][0]=="Rule4 "  and list_tuple[3][0]=="Rule5 ") :
                        lis1=str(list_tuple[1][1][1]).split()
                        lis2=str(list_tuple[3][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            list_tuple.remove(list_tuple[3])

            elif (list_tuple[0][0]=="Rule2 "  and list_tuple[1][0]=="Rule3 "  and list_tuple[2][0]=="Rule4 "  and list_tuple[3][0]=="Rule5 ") :
                        lis1=str(list_tuple[1][1][1]).split()
                        lis2=str(list_tuple[3][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            list_tuple.remove(list_tuple[3])

            elif (list_tuple[0][0]=="Rule1 "  and list_tuple[1][0]=="Rule2 "  and list_tuple[2][0]=="Rule3 "  and list_tuple[3][0]=="Rule5 ") :
                        lis1=str(list_tuple[2][1][1]).split()
                        lis2=str(list_tuple[3][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            list_tuple.remove(list_tuple[3])
            
            elif (list_tuple[0][0]=="Rule1 "  and list_tuple[1][0]=="Rule2 " and list_tuple[2][0]=="Rule3 " and list_tuple[3][0]=="Rule6 ") :
                        lis1=str(list_tuple[2][1][1]).split()
                        lis2=str(list_tuple[3][1][1]).split()
                        if str(lis1[0]) == str(lis2[0]):
                            print(lis1[0]+" "+lis2[1]+" "+lis1[1])
                            list_tuple.remove(list_tuple[2])
                            list_tuple.remove(list_tuple[3])
            
            elif (list_tuple[0][0]=="Rule1 "  and list_tuple[1][0]=="Rule3 " and list_tuple[2][0]=="Rule4 " and list_tuple[3][0]=="Rule6 ") :
                        lis1=str(list_tuple[1][1][1]).split()
                        lis2=str(list_tuple[3][1][1]).split()
                        if str(lis1[0]) == str(lis2[0]):
                            print(lis1[0]+" "+lis2[1]+" "+lis1[1])
                            list_tuple.remove(list_tuple[1])
                            list_tuple.remove(list_tuple[3])
                            
            elif (list_tuple[0][0]=="Rule1 "  and list_tuple[1][0]=="Rule3 " and list_tuple[2][0]=="Rule5 " and list_tuple[3][0]=="Rule6 ") :
                        lis1=str(list_tuple[1][1][1]).split()
                        lis2=str(list_tuple[3][1][1]).split()
                        if str(lis1[0]) == str(lis2[0]):
                            print(lis1[0]+" "+lis2[1]+" "+lis1[1])
                            list_tuple.remove(list_tuple[1])
                            list_tuple.remove(list_tuple[3])
                            
            elif (list_tuple[0][0]=="Rule2 "  and list_tuple[1][0]=="Rule3 " and list_tuple[2][0]=="Rule5 " and list_tuple[3][0]=="Rule6 ") :
                        lis1=str(list_tuple[1][1][1]).split()
                        lis2=str(list_tuple[3][1][1]).split()
                        if str(lis1[0]) == str(lis2[0]):
                            print(lis1[0]+" "+lis2[1]+" "+lis1[1])
                            list_tuple.remove(list_tuple[1])
                            list_tuple.remove(list_tuple[3])
                            
                            
            elif (list_tuple[0][0]=="Rule2 "  and list_tuple[1][0]=="Rule3 " and list_tuple[2][0]=="Rule4 " and list_tuple[3][0]=="Rule6 ") :
                        lis1=str(list_tuple[1][1][1]).split()
                        lis2=str(list_tuple[3][1][1]).split()
                        if str(lis1[0]) == str(lis2[0]):
                            print(lis1[0]+" "+lis2[1]+" "+lis1[1])
                            list_tuple.remove(list_tuple[1])
                            list_tuple.remove(list_tuple[3])
                            
            elif (list_tuple[0][0]=="Rule3 "  and list_tuple[1][0]=="Rule4 " and list_tuple[2][0]=="Rule5 " and list_tuple[3][0]=="Rule6 ") :
                        lis1=str(list_tuple[0][1][1]).split()
                        lis2=str(list_tuple[3][1][1]).split()
                        if str(lis1[0]) == str(lis2[0]):
                            print(lis1[0]+" "+lis2[1]+" "+lis1[1])
                            list_tuple.remove(list_tuple[0])
                            list_tuple.remove(list_tuple[3])

def _3_rules(list_tuple=[]): 
       

            if (list_tuple[0][0]=="Rule1 "  and list_tuple[1][0]=="Rule2 "  and list_tuple[2][0]=="Rule3 ") :
                        lis1=str(list_tuple[1][1][1]).split()
                        lis2=str(list_tuple[2][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            list_tuple.remove(list_tuple[2])


            elif (list_tuple[0][0]=="Rule1 "  and list_tuple[1][0]=="Rule2 "  and list_tuple[2][0]=="Rule5 "):
                        lis1=str(list_tuple[1][1][1]).split()
                        lis2=str(list_tuple[2][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            list_tuple.remove(list_tuple[2])

            elif (list_tuple[0][0]=="Rule2 "  and list_tuple[1][0]=="Rule3 "  and list_tuple[2][0]=="Rule5 "):
                        lis1=str(list_tuple[0][1][1]).split()
                        lis2=str(list_tuple[2][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            list_tuple.remove(list_tuple[2])

            elif (list_tuple[0][0]=="Rule2 "  and list_tuple[1][0]=="Rule4 "  and list_tuple[2][0]=="Rule5 "):
                        lis1=str(list_tuple[0][1][1]).split()
                        lis2=str(list_tuple[2][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            list_tuple.remove(list_tuple[2])

            elif (list_tuple[0][0]=="Rule1 "  and list_tuple[1][0]=="Rule3 "  and list_tuple[2][0]=="Rule5 ") :
                        lis1=str(list_tuple[1][1][1]).split()
                        lis2=str(list_tuple[2][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            list_tuple.remove(list_tuple[2])

            elif (list_tuple[0][0]=="Rule2 "  and list_tuple[1][0]=="Rule3 "  and list_tuple[2][0]=="Rule5 ") :
                        lis1=str(list_tuple[1][1][1]).split()
                        lis2=str(list_tuple[2][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            list_tuple.remove(list_tuple[2])

            elif (list_tuple[0][0]=="Rule3 "  and list_tuple[1][0]=="Rule4 "  and list_tuple[2][0]=="Rule5 ") :
                        lis1=str(list_tuple[0][1][1]).split()
                        lis2=str(list_tuple[2][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            list_tuple.remove(list_tuple[2])
                            
            elif (list_tuple[0][0]=="Rule1 "  and list_tuple[1][0]=="Rule3 "  and list_tuple[2][0]=="Rule6 ") :
                        lis1=str(list_tuple[1][1][1]).split()
                        lis2=str(list_tuple[2][1][1]).split()
                        if str(lis1[0]) == str(lis2[0]):
                            print(lis1[0]+" "+lis2[1]+" "+lis1[1])
                            list_tuple.remove(list_tuple[1])
                            list_tuple.remove(list_tuple[2])
                            
            elif (list_tuple[0][0]=="Rule2 "  and list_tuple[1][0]=="Rule3 "  and list_tuple[2][0]=="Rule6 ") :
                        lis1=str(list_tuple[1][1][1]).split()
                        lis2=str(list_tuple[2][1][1]).split()
                        if str(lis1[0]) == str(lis2[0]):
                            print(lis1[0]+" "+lis2[1]+" "+lis1[1])
                            list_tuple.remove(list_tuple[1])
                            list_tuple.remove(list_tuple[2])
                            
            elif (list_tuple[0][0]=="Rule3 "  and list_tuple[1][0]=="Rule4 "  and list_tuple[2][0]=="Rule6 ") :
                        lis1=str(list_tuple[0][1][1]).split()
                        lis2=str(list_tuple[2][1][1]).split()
                        if str(lis1[0]) == str(lis2[0]):
                            print(lis1[0]+" "+lis2[1]+" "+lis1[1])
                            list_tuple.remove(list_tuple[0])
                            list_tuple.remove(list_tuple[2])
                            
            elif (list_tuple[0][0]=="Rule3 "  and list_tuple[1][0]=="Rule5 "  and list_tuple[2][0]=="Rule6 ") :
                        lis1=str(list_tuple[0][1][1]).split()
                        lis2=str(list_tuple[2][1][1]).split()
                        if str(lis1[0]) == str(lis2[0]):
                            print(lis1[0]+" "+lis2[1]+" "+lis1[1])
                            list_tuple.remove(list_tuple[0])
                            list_tuple.remove(list_tuple[2])
                            
                            
def _2_rules(list_tuple=[]): 
    

            if (list_tuple[0][0]=="Rule1 "  and list_tuple[1][0]=="Rule2 ") :
                        lis=str(list_tuple[1][1][1]).split()
                        if str(list_tuple[0][1][1]) == str(lis[0]):
                            list_tuple.remove(list_tuple[0])

            elif (list_tuple[0][0]=="Rule2 "  and list_tuple[1][0]=="Rule3 ") :
                        lis1=str(list_tuple[0][1][1]).split()
                        lis2=str(list_tuple[1][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            list_tuple.remove(list_tuple[1])

            elif (list_tuple[0][0]=="Rule2 "  and list_tuple[1][0]=="Rule5 ") :
                        lis1=str(list_tuple[0][1][1]).split()
                        lis2=str(list_tuple[1][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            list_tuple.remove(list_tuple[1])

            elif (list_tuple[0][0]=="Rule3 "  and list_tuple[1][0]=="Rule5 ") :
                        lis1=str(list_tuple[0][1][1]).split()
                        lis2=str(list_tuple[1][1][1]).split()
                        if str(lis1[1]) == str(lis2[1]):
                            list_tuple.remove(list_tuple[1])
                            
            elif (list_tuple[0][0]=="Rule3 "  and list_tuple[1][0]=="Rule6 ") :
                        lis1=str(list_tuple[0][1][1]).split()
                        lis2=str(list_tuple[1][1][1]).split()
                        if str(lis1[0]) == str(lis2[0]):
                            list_tuple.remove(list_tuple[1])
                            




# pronoun resolution section 

In [491]:
p=["he","she","her","his","him","they","their","them","us","we","our","me","my","you","your","i"]  
num_of_sentence_for_pronoun_resolution=2

# getting corference clusters 
def pronoun_resolution_allen(sent,predictor=predictor):
    
    pred = predictor.predict(
        document= sent
    )

    clusters = pred['clusters']
    document = pred['document']
    n = 0
    doc = {}
    for obj in document:
        doc.update({n :  obj}) #what I'm doing here is creating a dictionary of each word with its respective index, making it easier later.
        n = n+1

    clus_all = []
    cluster = []
    clus_one = {}
    s=""
    for i in range(0, len(clusters)):
        one_cl = clusters[i]
        for count in range(0, len(one_cl)):
            obj = one_cl[count]            
            for num in range((obj[0]), (obj[1]+1)):                
                for n in doc:
                    if num == n:
                        if obj[1]-obj[0]>0:
                            s+=" "+str(doc[n])
                            if len(s.split())>=2 :
                                cluster.append(s)
                        elif obj[1]-obj[0]==0: 
                            cluster.append(doc[n])
        clus_all.append(cluster)
        cluster = [] 
    return clus_all
    
    


#for checking if the results of pronoun resolution need more resolution like
#my sister yesterday comes , she won awards (the results of subject is she .... after pronoun resolution we get my sister ... but until now we dont know 
#which the real subject so we use this function to again make pronoun resolution using my rule )
def check_need_for_more_resol(element,dialog_number,index_of_sentence,sayers_of_dialog):
    count = element.count(' and ')                 
    while (count >=0 ):                     
        if " my " in element :
            subb=my_pronoun_resolution_for_i(element," ",dialog_number,index_of_sentence,sayers_of_dialog)
            element=element.replace(" my "," "+str(subb)+" ")

        elif element.endswith(' my')==True:
            subb=my_pronoun_resolution_for_i(element," ",dialog_number,index_of_sentence,sayers_of_dialog)
            element=element.replace(" my"," "+str(subb))

        elif " i " in element :
            subb=my_pronoun_resolution_for_i(element," ",dialog_number,index_of_sentence,sayers_of_dialog)
            element=element.replace(" i "," "+str(subb)+" ")
            print("element .... ",element)

        elif  element.endswith(' i')==True:
            subb=my_pronoun_resolution_for_i(element," ",dialog_number,index_of_sentence,sayers_of_dialog)
            element=element.replace(" i"," "+str(subb))
            print("element .... ",element)

        elif " me " in element :
            subb=my_pronoun_resolution_for_i(element," ",dialog_number,index_of_sentence,sayers_of_dialog)
            element=element.replace(" me "," "+str(subb)+" ")
            
        elif element.endswith(' me')==True:
            subb=my_pronoun_resolution_for_i(element," ",dialog_number,index_of_sentence,sayers_of_dialog)
            element=element.replace(" me"," "+str(subb))


        elif " you " in element :
            subb=my_pronoun_resolution_for_you(element," ",dialog_number,index_of_sentence,sayers_of_dialog)
            element=element.replace(" you "," "+str(subb)+" ")
            
        elif element.endswith(' you')==True:
            subb=my_pronoun_resolution_for_you(element," ",dialog_number,index_of_sentence,sayers_of_dialog)
            element=element.replace(" you"," "+str(subb))


        elif " your " in element :
            subb=my_pronoun_resolution_for_you(element," ",dialog_number,index_of_sentence,sayers_of_dialog)
            element=element.replace(" your "," "+str(subb)+" ")
            
        elif element.endswith(' your')==True:
            subb=my_pronoun_resolution_for_you(element," ",dialog_number,index_of_sentence,sayers_of_dialog)
            element=element.replace(" your"," "+str(subb))

        elif (" my " not in element )and (" i " not in element) and (" me " not in element) and (" your " not in element) and (" you " not in element) and(element.endswith(' you')==False) and(element.endswith(' your')==False) and(element.endswith(' i')==False) and(element.endswith(' my')==False) and(element.endswith(' me')==False):
            element=" "
            
        count-=1
        
    return element            



# my rules for pronoun resolution for i
def my_pronoun_resolution_for_i(sub,sub2,dialog_number,index_of_sentence,sayers_of_dialog):
    subjects=""
    if str(sub2)==" ":
        subjects=sayers_of_dialog[dialog_number][index_of_sentence]
    elif str(sub2) !=" ":
        c=sayers_of_dialog[dialog_number][index_of_sentence]+" "+str(sub2)
        subjects=c
    return subjects
        
    
    
# my rules for pronoun resolution for you
def my_pronoun_resolution_for_you(sub,sub2,dialog_number,index_of_sentence,sayers_of_dialog,len_of_dialog):
    subjects=""
    if len_of_dialog[dialog_number]-1==index_of_sentence:
        if sayers_of_dialog[dialog_number][index_of_sentence-1]!=sayers_of_dialog[dialog_number][index_of_sentence]:
            if str(sub2)==" ": 
                subjects=sayers_of_dialog[dialog_number][index_of_sentence-1]
            elif str(sub2)!=" ":
                c=sayers_of_dialog[dialog_number][index_of_sentence-1]+" "+str(sub2)
                subjects=c
    if len_of_dialog[dialog_number]-1>index_of_sentence:
        if sayers_of_dialog[dialog_number][index_of_sentence+1]!=sayers_of_dialog[dialog_number][index_of_sentence]:
            if str(sub2)==" ": 
                subjects=sayers_of_dialog[dialog_number][index_of_sentence+1]
            elif str(sub2)!=" ": 
                c=sayers_of_dialog[dialog_number][index_of_sentence+1]+" "+str(sub2)
                subjects=c
    return subjects
            
            
def my_pronoun_resolution_rules_for_we(sub,sub2,dialog_number,index_of_sentence,sayers_of_dialog,len_of_dialog):
    subjects=""
    if len_of_dialog[dialog_number]-1==index_of_sentence:
        if sayers_of_dialog[dialog_number][index_of_sentence-1]!=sayers_of_dialog[dialog_number][index_of_sentence]:
            if str(sub2)==" ": 
                c=sayers_of_dialog[dialog_number][index_of_sentence-1]+" and "+sayers_of_dialog[dialog_number][index_of_sentence]
                subjects=c
            elif str(sub2)!=" ":
                c=sayers_of_dialog[dialog_number][index_of_sentence-1]+" and "+sayers_of_dialog[dialog_number][index_of_sentence]+" "+str(sub2)
                subjects=c

    if len_of_dialog[dialog_number]-1>index_of_sentence:
        if sayers_of_dialog[dialog_number][index_of_sentence+1]!=sayers_of_dialog[dialog_number][index_of_sentence]:
            if str(sub2)==" ": 
                c=sayers_of_dialog[dialog_number][index_of_sentence+1]+" and "+sayers_of_dialog[dialog_number][index_of_sentence]
                subjects=c
            elif str(sub2)!=" ": 
                c=sayers_of_dialog[dialog_number][index_of_sentence+1]+" and "+sayers_of_dialog[dialog_number][index_of_sentence]+" "+str(sub2)
                subjects=c
    return subjects
    
    
    
    
def check_type_of_subject(sub):
    
    
    if isinstance(sub, tuple)==False:
        return sub," "
    if isinstance(sub, tuple)==True:
        return sub[0],sub[1]
    


def getting_n_previous_sentences(index_of_sentence,num_of_sentence_for_pronoun_resolution,sentences_of_dialogs,dialog_number):
    sentence=[]
    text=""
    index_of_sentence1=index_of_sentence
    while(index_of_sentence1>=0 and num_of_sentence_for_pronoun_resolution>=0):
        sentence.append(sentences_of_dialogs[dialog_number][index_of_sentence1])
        index_of_sentence1-=1
        num_of_sentence_for_pronoun_resolution-=1
    for i in range(len(sentence)):
        text+=" "+str(sentence[len(sentence)-i-1])
    return text    
    


In [492]:
    
    
def pronoun_resolution_for_all_cases_of_pronouns(aggrigation_list=[],dialog_number=0,index_of_sentence=0,sentences_of_dialogs=[],num_of_sentence_for_pronoun_resolution=3,sayers_of_dialog=[],len_of_dialog=[]):
    b=aggrigation_list
    
    if len(b[0]) !=0 and  len(b[1]) !=0:
        subjects=[]
        
        for i in range(len(b[1])):
            
            sub,sub2=check_type_of_subject(b[1][i])

            if str(sub)=="i" or str(sub)=="my" or str(sub)=="me":
                subjects.append(my_pronoun_resolution_for_i(sub,sub2,dialog_number,index_of_sentence,sayers_of_dialog))

                
            elif str(sub)=="you" or str(sub)=="your" :
                subjects.append(my_pronoun_resolution_for_you(sub,sub2,dialog_number,index_of_sentence,sayers_of_dialog,len_of_dialog))

                
            elif str(sub)=="her" or str(sub)=="she" :
                
                text=getting_n_previous_sentences(index_of_sentence,num_of_sentence_for_pronoun_resolution,sentences_of_dialogs,dialog_number)
                
                print("....... more than one sentnce for making pro resolution ....... (", (text) ,")","\n" )    
                
                resol=pronoun_resolution_allen(text)

                if len(resol)!=0:
                    for i in resol:
                        if "her" in i or "she" in i :
                            for element in i :
                                if len(element.split())==1:
                                    if element not in p:
                                        if str(sub2)==" ": 
                                            subjects.append(element)
                                        elif str(sub2)!=" ": 
                                            c=element+" "+str(sub2)
                                            subjects.append(c)
                                elif len(element.split())==2:
                                    ele=check_need_for_more_resol(element,dialog_number,index_of_sentence,sayers_of_dialog)
                                    #like my sister after pronoun resolution should make pronoun again using my rules 
                                    if ele==" ":
                                        if str(sub2)==" ": 
                                            subjects.append(element)
                                        elif str(sub2)!=" ":
                                            c=element+" "+str(sub2)
                                            subjects.append(c)
                                    else:
                                        if str(sub2)==" ": 
                                            subjects.append(ele)
                                        elif str(sub2)!=" ":
                                            c=ele+" "+str(sub2)
                                            subjects.append(c)
                                        
                                    
                                    



            elif str(sub)=="he" or str(sub)=="his" or str(sub)=="him" :
                
                text=getting_n_previous_sentences(index_of_sentence,num_of_sentence_for_pronoun_resolution,sentences_of_dialogs,dialog_number)
                
                print("............... more than one sentnce for making pro resolution ...............(", (text) ,")" ,"\n" )    
                
                resol=pronoun_resolution_allen(text)

                if len(resol)!=0:
                    for i in resol:
                        if "his" in i or "he" in i  or "him" in i :
                            for element in i :
                                if len(element.split())==1:
                                    if element not in p:
                                        if str(sub2)==" ": 
                                            subjects.append(element)
                                        elif str(sub2)!=" ": 
                                            c=element+" "+str(sub2)
                                            subjects.append(c)
                                elif len(element.split())==2:
                                    #like my sister after pronoun resolution should make pronoun again using my rules 
                                    
                                    ele=check_need_for_more_resol(element,dialog_number,index_of_sentence,sayers_of_dialog)
                                    
                                    if ele==" ":   
                                        if str(sub2)==" ": 
                                            subjects.append(element)
                                        elif str(sub2)!=" ":
                                            c=element+" "+str(sub2)
                                            subjects.append(c)
                                    else:
                                        if str(sub2)==" ": 
                                            subjects.append(ele)
                                        elif str(sub2)!=" ":
                                            c=ele+" "+str(sub2)
                                            subjects.append(c)
                                        
                                    


            
            elif str(sub)=="they" or str(sub)=="their" :
                
                text=getting_n_previous_sentences(index_of_sentence,num_of_sentence_for_pronoun_resolution,sentences_of_dialogs,dialog_number)
                
                print("............... more than one sentnce for making pro resolution ............... (", (text) ,")","\n" )    
                
                resol=pronoun_resolution_allen(text)
                
                if len(resol)!=0:
                    for i in resol:
                        for element in i :
                            if (" and " in element) and (" they " not in element) and (" their " not in element) and (element.endswith('and')==False ):
                                if element.endswith('they')==False and element.endswith('their')==False:
                                    if len(element.split())>=3:
                                        ele=check_need_for_more_resol(element,dialog_number,index_of_sentence,sayers_of_dialog)
                                            
                                        if ele ==" ":
                                            if str(sub2)==" ": 
                                                subjects.append(element)
                                            elif str(sub2)!=" ": 
                                                c=element+" "+str(sub2)
                                                subjects.append(c)
                                        else:
                                            if str(sub2)==" ": 
                                                subjects.append(ele)
                                            elif str(sub2)!=" ": 
                                                c=ele+" "+str(sub2)
                                                subjects.append(c)


            elif str(sub)=="we" or str(sub)=="our" or str(sub)=="us":
                
                text=getting_n_previous_sentences(index_of_sentence,num_of_sentence_for_pronoun_resolution,sentences_of_dialogs,dialog_number)
                
                    
                print("............... more than one sentnce for making pro resolution ............... (", (text) ,")" ,"\n" )    
                
                resol=pronoun_resolution_allen(text)

                if len(resol)!=0 :
                    for i in resol:
                        for element in i :
                            if (" and " in element) and (" we " not in element) and (" our " not in element ) and (" us " not in element ) and element.endswith('and')==False :
                                if element.endswith('we')==False and element.endswith('our')==False and element.endswith('us')==False:
                                    if len(element.split())>=3:
                                        ele=check_need_for_more_resol(element,dialog_number,index_of_sentence,sayers_of_dialog)

                                        if ele==" ":
                                            if str(sub2)==" ": 
                                                subjects.append(element)
                                            elif str(sub2)!=" ": 
                                                c=element+" "+str(sub2)
                                                subjects.append(c)
                                        else:
                                            if str(sub2)==" ": 
                                                subjects.append(ele)
                                            elif str(sub2)!=" ": 
                                                c=ele+" "+str(sub2)
                                                subjects.append(c)
                                            
                if len(subjects)==0:
                    subjects.append(my_pronoun_resolution_rules_for_we(sub,sub2,dialog_number,index_of_sentence,sayers_of_dialog,len_of_dialog))
                    

        subjects = list(pd.Series(subjects).drop_duplicates())  
        return subjects
    


# aggregation the results of pronoun resolution and the results of all previous steps 

In [493]:
def aggregation_between_filtering_and_resolution(aggregation_for_sentence,resolution):
            agg=[]
            b=aggregation_for_sentence
            if resolution !=None and len(resolution)==0 and (len(b)==2):
                agg.append(b)
            if resolution !=None and len(resolution)!=0 and (len(b)==2):
                if b[0]!=0 and b[1]!=0:
                    if len(b[1])==1:
                        if isinstance(b[1][0], tuple)==True:
                            agg.append((b[0] ,resolution))
                        elif isinstance(b[1][0], tuple)==False:
                            if str(b[1][0]) in p :
                                agg.append((b[0] ,resolution))
                            elif str(b[1][0]) not in p :
                                agg.append((b[0] ,b[1]))
                    elif len(b[1])>1:
                        if len(b[1])==len(resolution):
                            agg.append((b[0] ,resolution))
                            
                        elif len(b[1])!=len(resolution):
                            for i in range(len(b[1])):
                                if i+1 < len(b[1]): 
                                    if equal_subjects(b[1][i],b[1][i+1]):
                                        b[1].remove(b[1][i])
                            for i in range(len(resolution)):
                                for j in range(len(b[1])):
                                    if isinstance(b[1][j] , tuple)==False:
                                        if str(b[1][j]) in p :
                                            b[1][j]=resolution[i]
                                            continue
                                    elif isinstance(b[1][j] , tuple)==True:
                                        c=b[1][j]
                                        if str(c[0]) in p :
                                            
                                            b[1][j]=resolution[i]
                                            
                                            continue
                            agg.append((b[0] ,b[1]))           
            return agg    

In [494]:
import pandas
output=pandas.read_csv('labels.csv') 



# checking all previous steps (extract events , and subjects with pronoun resolution) , applaying filtering and aggregation functions

In [507]:
f = open("Dialogs_data.txt", "r")

def splitting_sayers_and_sentences():
    sentence=[]
    sayers=[]
    for line in f:
        stripped_line = line.strip()
        sayer=stripped_line[:stripped_line.find(":")]
        sayers.append(sayer)
        stripped_line=stripped_line[stripped_line.find(":")+1:]
        sentence.append(stripped_line)    
    return sayers,sentence

sayers,sentence = splitting_sayers_and_sentences() 


def get_sayers_of_all_dialogs(sayers):
    sayers_of_all_dialogs=[]
    sy=[]
    for i in sayers :
        if i!="":
            sy.append(i)
        if i=="" and len(sy)!=0 :
            sayers_of_all_dialogs.append(sy)
            sy=[]
    return sayers_of_all_dialogs
            
    
sayers_of_all_dialogs = get_sayers_of_all_dialogs(sayers)        
            
    
def get_len_of_all_dialogs(sayers):
    len_of_all_dialogs=[]
    n=0
    for i in sayers :
        if i!="":
            n+=1
        if i=="" and len_of_all_dialogs!=0:
            len_of_all_dialogs.append(n)
            n=0
    return len_of_all_dialogs


len_of_all_dialogs = get_len_of_all_dialogs(sayers)
            
        
def get_sentences_of_all_dialogs(sentence):
    sentences_of_all_dialogs=[]
    sentences_of_each_dialog=[]
    for i in sentence :
        if i!="":
            sentences_of_each_dialog.append(i)
        if i=="":
            sentences_of_all_dialogs.append(sentences_of_each_dialog)
            sentences_of_each_dialog=[]
    return sentences_of_all_dialogs    

sentences_of_all_dialogs = get_sentences_of_all_dialogs(sentence)


def get_final_results(sentence):
    index=-1
    subject_result_befor_resolution=[]
    final_results=[]   
    #counter for index of sentences


    index_of_sentence_in_dialog=[]
    index_of_sentences_in_all_dialogs=[]
    #  dialog number 
    n=0

    aggreigation_dialog=[]
    
    for j,i in enumerate(sentence) :
        print(sayers[j]," ",i)
        list_tuple=[]
        t=i.lower()
        doc=nlp(t)

        if i!="":
            index+=1

            index_of_sentence_in_dialog.append(index)

        p1=Rules(doc, verbs_without_need_objects)
        text=p1.retrival_senetce_rule1()
        if text!=None:
            list_tuple.append(["Rule1 ",text , findSubs_for_verbEvents(nlp(text[0]),verbs_without_need_objects)])


        p2=Rules(doc, verbs_with_objects)
        text=p2.retrival_senetce_rule2()
        if text!=None:
            list_tuple.append(["Rule2 ",text ,findSubs_for_verbEvents(nlp(text[0]),verbs_with_objects)])    

        p3=Rules(doc, nouns)
        text=p3.retrival_senetce_rule3()
        if text!=None:
            list_tuple.append(["Rule3 ",text ,findSubs_for_nounsEvents(nlp(text[0]),nouns)])


        p4=Rules(doc, adjectives)
        text=p4.retrival_senetce_rule4()
        if text!=None:
            list_tuple.append(["Rule4 ",text ,findSubs_for_adjectiveEvents(nlp(text[0]),adjectives)])


        p5=Rules(doc, noun_direct_relation)
        text=p5.retrival_senetce_rule5()
        if text!=None:
            list_tuple.append(["Rule5 ",text , findSubs_for_direct_relation(nlp(text[0]),noun_direct_relation)])


        p6=Rules(doc, verb_with_prepo)
        text=p6.retrival_senetce_rule6()
        if text!=None:
            list_tuple.append(["Rule6 ",text ,findSubs_for_actionEvents(nlp(text[0]),verb_with_prepo)])
        # for rules conflict
        filtering(list_tuple)
        # for aggregation all subjects or all events for the sentence 
        if aggregation_for_sentence(list_tuple)!=None:

            if aggregation_for_sentence(list_tuple)[0]!="":
                b=aggregation_for_sentence(list_tuple)

                if len(b[1])!=0:
                    id__=str(n+1)+"_"+str(index)

                    print("...............subject result befor resolution..............",b[1],"\n" )
                    subject_result_befor_resolution.append((b[1],id__))

                # make pronoun resolution for getting the real person name    
                resolution=pronoun_resolution_for_all_cases_of_pronouns(b,n,index,sentences_of_all_dialogs,num_of_sentence_for_pronoun_resolution,sayers_of_all_dialogs,len_of_all_dialogs)
                # for aggregating the results of filtering and aggregating sentence with the results of pronoun resolution
                aggrigation=aggregation_between_filtering_and_resolution(b,resolution)

                if len(aggrigation)!=0:
                    print(" ..................the final results.................... ",aggrigation,"\n" )
                    id_=str(n+1)+"_"+str(index)
                    final_results.append((aggrigation,id_))



        if i=="" and len(index_of_sentence_in_dialog)!=0 :
         



            index_of_sentences_in_all_dialogs.append(index_of_sentence_in_dialog)
            index=-1

            index_of_sentence_in_dialog=[]
            n+=1
            print("\n")
    return  final_results 

final_results=get_final_results(sentence)
    


W0731 23:53:36.985095 28844 warnings.py:110] C:\Users\Windows dunya\Anaconda3\lib\site-packages\ipykernel_launcher.py:164: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



Mark   Susan is going to get a divorce .
...............subject result befor resolution.............. [susan] 

 ..................the final results....................  [(['get divorce'], [susan])] 

Jhon   How do you know that ?
Mark   She told me that Peter and she has a quarrel last nigh , and she left this morning , bag and baggage .
Jhon   I see . But I think you are making a fuss .
   


Mark   these days too many people are getting divorced .


W0731 23:53:43.505668 28844 warnings.py:110] C:\Users\Windows dunya\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



Jhon   If they live together , then at least they're finding out if they're really compatible or not .
   


Jhon   what is happend with Mari and george ?
Mark   They got a divorce at last .
...............subject result befor resolution.............. [they] 

............... the 3 sentences for making pro resolution ............... (  what is happend with Mari and george ? They got a divorce at last . ) 

 ..................the final results....................  [(['got divorce'], [' Mari and george'])] 

Jhon   It's inevitable .
Mark   Their love was built on the sand , and this is why their marriage has landed on the rocks.
   


Jhon   what is happend with Mari?
Mark   I heard she has passed the bar exam and married recently .
...............subject result befor resolution.............. [she] 

....... the 3 sentences for making pro resolution ....... (  what is happend with Mari? I heard she has passed the bar exam and married recently . ) 

 ..................the final results...

# preprocessing the results of our task and the labeled dataset for comparing results and evaluating them

In [438]:
# my results events and subjects 
def get_sub_event_id_from_result(final_results):
    dialog_id_results = []
    events_results = []
    subjects_results = []
    for i in final_results:
        b = i[0]
        print(b)
        dialog_id_results.append(i[1])
        print("events ==> ", str(b[0][0]), " ||| subjects ==> ", str(b[0][1]), " ||| dialog number ==> ", i[1])
        print("\n")
        events_results.append(b[0][0])
        subjects_results.append(b[0][1])
    return dialog_id_results, events_results, subjects_results

dialog_id_results, events_results, subjects_results = get_sub_event_id_from_result(final_results)



         

[(['get divorce'], [susan])]
events ==>  ['get divorce']  ||| subjects ==>  [susan]  ||| dialog number ==>  1_0


[(['got divorce'], [' Mari and george'])]
events ==>  ['got divorce']  ||| subjects ==>  [' Mari and george']  ||| dialog number ==>  3_1


[(['married', 'passed exam'], ['Mari'])]
events ==>  ['married', 'passed exam']  ||| subjects ==>  ['Mari']  ||| dialog number ==>  4_1


[(['had wedding'], ['She'])]
events ==>  ['had wedding']  ||| subjects ==>  ['She']  ||| dialog number ==>  4_3


[(['married'], ['Jhon and Abigail'])]
events ==>  ['married']  ||| subjects ==>  ['Jhon and Abigail']  ||| dialog number ==>  6_0


[(['your wedding'], ['Abigail'])]
events ==>  ['your wedding']  ||| subjects ==>  ['Abigail']  ||| dialog number ==>  7_0


[(['married'], ['Abigail husband', 'Abigail'])]
events ==>  ['married']  ||| subjects ==>  ['Abigail husband', 'Abigail']  ||| dialog number ==>  7_1


[(['your wedding'], ['Abigail'])]
events ==>  ['your wedding']  ||| subjects ==>  ['Ab

In [439]:


#getting events and subjects from labeled dataset
def get_sub_event_id_from_labeledData(output):
    events=[]
    subjects=[]
    dialog_id=[]
    subjects_no_resolution=[]
    for i in range(len(output)):
        events.append(output["events"][i])
        subjects.append(output["subjects"][i])
        dialog_id.append(output["dialog_id"][i])
        subjects_no_resolution.append(output["subjects_no_resolution"][i])
    for i in range(len(events)):
        print("events ==> ",events[i], " ||| subjects ==> ",subjects[i]," ||| subjects befor resolution ==> ",subjects_no_resolution[i]," ||| dialog number ==> ",dialog_id[i])
        print("\n")
    return dialog_id,events,subjects,subjects_no_resolution

dialog_id,events,subjects,subjects_no_resolution = get_sub_event_id_from_labeledData(output)


events ==>  get divorce  ||| subjects ==>  Susan  ||| subjects befor resolution ==>  Susan  ||| dialog number ==>  1_0


events ==>  None  ||| subjects ==>  None  ||| subjects befor resolution ==>  None  ||| dialog number ==>  2


events ==>  got divorce  ||| subjects ==>  Mari,george  ||| subjects befor resolution ==>  they  ||| dialog number ==>  3_1


events ==>  married,passed exam  ||| subjects ==>  Mari  ||| subjects befor resolution ==>  she  ||| dialog number ==>  4_1


events ==>  had wedding  ||| subjects ==>  Mari  ||| subjects befor resolution ==>  she  ||| dialog number ==>  4_3


events ==>  None  ||| subjects ==>  None  ||| subjects befor resolution ==>  None  ||| dialog number ==>  5


events ==>  married  ||| subjects ==>  Abigail,Abigail husband  ||| subjects befor resolution ==>  we  ||| dialog number ==>  6_0


events ==>  your wedding  ||| subjects ==>  Abigail  ||| subjects befor resolution ==>  your  ||| dialog number ==>  7_0


events ==>  married  ||| subjects 

In [440]:

# preprocessing on labeled dataset for easier matching with results



def preprocessing_on_labeledData(events,subjects,subjects_no_resolution):
    events_preprocessed = []
    subjects_preprocessed = []
    subjects_no_resolution_preprocessed = []
    for i, j in enumerate(events):

        # sentence has more one events like 'passed exam,married' transformed to [passed exam,married]
        if "," in str(events[i]):
            b = []
            c = events[i]
            b.append(c[:c.find(",")])
            b.append(c[c.find(",") + 1:])

            events_preprocessed.append(b)
        if "," not in str(events[i]):
            events_preprocessed.append([events[i]])

        # the same for subjects after resolution
    for i, j in enumerate(subjects):
        if "," in str(subjects[i]):
            d = []
            c = subjects[i]
            d.append(c[:c.find(",")])
            d.append(c[c.find(",") + 1:])

            subjects_preprocessed.append(d)
        if "," not in str(subjects[i]):
            subjects_preprocessed.append([subjects[i]])

        # the same for subjects befor resolution
    for i, j in enumerate(subjects_no_resolution):
        if "," in str(subjects_no_resolution[i]):
            e = []
            c = subjects_no_resolution[i]
            e.append(c[:c.find(",")])
            e.append(c[c.find(",") + 1:])

            subjects_no_resolution_preprocessed.append(e)

        if "," not in str(subjects_no_resolution[i]):
            subjects_no_resolution_preprocessed.append([subjects_no_resolution[i]])
    return events_preprocessed,subjects_preprocessed,subjects_no_resolution_preprocessed

events_preprocessed,subjects_preprocessed,subjects_no_resolution_preprocessed = preprocessing_on_labeledData(events,subjects,subjects_no_resolution)



# all this for subjects results after pronoun resolution

# we and they resolution maybe get subjects like "yara and hadia" will transformed to ['yara','hadia']



In [445]:
def preprocessing_on_resultSubjects(subjects_results):
    subjects_results_preprocessed = []
    for i, j in enumerate(subjects_results):
        if " and " in str(subjects_results[i]):
            b = []
            c = subjects_results[i][0]
            b.append(c[:c.find(" and ")])
            b.append(c[c.find(" and ") + 5:])
            b.sort()
            subjects_results_preprocessed.append(b)
        else:
            subjects_results_preprocessed.append(subjects_results[i])

        # subjects like joy's father .. my results return it like [('joy','father')] so will transformed to ['joy father']
        for k, l in enumerate(subjects_results[i]):
            if isinstance(subjects_results[i][k], tuple) == True:
                c=subjects_results[i][k]
                
                subjects_results_preprocessed[i][k] = str(c[0]) + " " + str(c[1])
                
            else:
                subjects_results_preprocessed[i][k]=subjects_results_preprocessed[i][k]
                
    return subjects_results_preprocessed
subjects_results_preprocessed = preprocessing_on_resultSubjects(subjects_results)



In [448]:
def get_subject_result_befor_resolution_withoutID(subject_result_befor_resolution):
    subject_result_befor_resolution_withoutID = []
    for i in subject_result_befor_resolution:
        b = i[0]
        subject_result_befor_resolution_withoutID.append(b)
    return subject_result_befor_resolution_withoutID

subject_result_befor_resolution_withoutID = get_subject_result_befor_resolution_withoutID(subject_result_befor_resolution)


print(subject_result_befor_resolution_withoutID)

[[susan], [they], [she], [she], [we], [your], [(my, husband), i], [your], [we], [i], [you], [you], [i], [you], [i], [(your, son)], [she], [she], [i], [i], [my], [you], [i], [he], [they], [frank], [i], [i], [i], [he], [i], [i], [i], [we], [your], [we], [my], [i], [i], [i], [i], [i], [we], [i], [i], [i], [our], [i], [we], [i], [(my, stomach), i], [he], [i], [i], [i], [you], [i], [you], [i], [i], [you], [i], [my], [you], [i], [i], [(my, daughter)], [me], [i], [i], ['Margaret', matt], [nathan, 'Kriss'], [tomas], [george, 'Mari'], [you], [i], [mario], [i], ['Jhon'], [tom], [i], [i], [bati], [we], [i], [i], [i], ['karla son'], [i], [she], [i], [tom], [tom], [you], [i], [(my, wife), i], [i], [i], [i], [i], [she], [you], [i], [i], [i], [i], [i], [i], [we], [dani], [i], [you], [tony], ['karolina son'], [i], [i], [i], [i], [i], [mira], [jim], [i], [ezabela], [mario], [his], [her], [our], [my], [my], [my], [she], [i], [i], [she], [luke, 'Tomas'], [i], ['joy father']]


In [451]:
def preprocessing_on_subject_result_befor_resolution_withoutID(subject_result_befor_resolution_withoutID):
    for i,j in enumerate(subject_result_befor_resolution_withoutID):        
        for k,l in enumerate(subject_result_befor_resolution_withoutID[i]):
            if k<=len(subject_result_befor_resolution_withoutID[i])   :

                if isinstance(l,tuple)==True:

                    subject_result_befor_resolution_withoutID[i][k]=str(l[0])+" "+str(l[1])
                else:  
                    subject_result_befor_resolution_withoutID[i][k]=subject_result_befor_resolution_withoutID[i][k]

                
    return subject_result_befor_resolution_withoutID


subject_result_befor_resolution_withoutID = preprocessing_on_subject_result_befor_resolution_withoutID(subject_result_befor_resolution_withoutID)

In [452]:
print(subject_result_befor_resolution_withoutID)

[[susan], [they], [she], [she], [we], [your], ['my husband', i], [your], [we], [i], [you], [you], [i], [you], [i], ['your son'], [she], [she], [i], [i], [my], [you], [i], [he], [they], [frank], [i], [i], [i], [he], [i], [i], [i], [we], [your], [we], [my], [i], [i], [i], [i], [i], [we], [i], [i], [i], [our], [i], [we], [i], ['my stomach', i], [he], [i], [i], [i], [you], [i], [you], [i], [i], [you], [i], [my], [you], [i], [i], ['my daughter'], [me], [i], [i], ['Margaret', matt], [nathan, 'Kriss'], [tomas], [george, 'Mari'], [you], [i], [mario], [i], ['Jhon'], [tom], [i], [i], [bati], [we], [i], [i], [i], ['karla son'], [i], [she], [i], [tom], [tom], [you], [i], ['my wife', i], [i], [i], [i], [i], [she], [you], [i], [i], [i], [i], [i], [i], [we], [dani], [i], [you], [tony], ['karolina son'], [i], [i], [i], [i], [i], [mira], [jim], [i], [ezabela], [mario], [his], [her], [our], [my], [my], [my], [she], [i], [i], [she], [luke, 'Tomas'], [i], ['joy father']]


In [455]:
#fill lists with None values with the same labeled data long
def fill_with_null(dialog_id):
    null_subject=[]
    null_subject_befor_resolution=[]
    null_events=[]
    for i,j in enumerate(dialog_id):
        null_subject.append([['None'],dialog_id[i]])
        null_events.append([['None'],dialog_id[i]])
        null_subject_befor_resolution.append([['None'],dialog_id[i]])
    return null_subject,null_events,null_subject_befor_resolution

null_subject,null_events,null_subject_befor_resolution = fill_with_null(dialog_id)

In [461]:
def fill_null_with_value_of_subjects_results(null_subject,subjects_results_preprocessing):

    for i,j in enumerate(null_subject):
        for k,l in enumerate(subjects_results_preprocessing):
            if str(dialog_id_results[k]).strip() == str(null_subject[i][1]).strip():
                null_subject[i][0]=subjects_results_preprocessing[k]
                null_events[i][0]=events_results[k]
                null_subject_befor_resolution[i][0]=subject_result_befor_resolution_withoutID[k]
    return null_subject,null_events,null_subject_befor_resolution
final_subjects_res_preprocessing,final_events_res_preprocessing,final_subject_befor_resolution_res_preprocessing = fill_null_with_value_of_subjects_results(null_subject,subjects_results_preprocessed)
print(final_subjects_res_preprocessing)

[[[susan], '1_0'], [['None'], '2'], [[' Mari', 'george'], '3_1'], [['Mari'], '4_1'], [['She'], '4_3'], [['None'], '5'], [['Abigail', 'Jhon'], '6_0'], [['Abigail'], '7_0'], [['Abigail husband', 'Abigail'], '7_1'], [['Abigail'], '8_0'], [['None'], '9_2'], [['Jhon', 'Mark'], '9_3'], [['None'], '10'], [['None'], '11'], [['Jhon'], '12_1'], [['Jhon'], '13_0'], [['Jhon'], '14_0'], [['Jhon'], '14_1'], [['None'], '15'], [['Abigail'], '16_1'], [['Abigail'], '17_1'], [['Margaret son'], '18_0'], [['Pamela'], '19_1'], [['Pamela'], '19_2'], [['None'], '20'], [['Mark'], '21_2'], [['Mark'], '22_0'], [['Mark'], '22_2'], [['Margaret'], '23_0'], [['Abigail'], '24_2'], [['george'], '25_1'], [[' Abigail', 'george'], '26_1'], [[frank], '27_0'], [['Mark'], '28_1'], [['Mark'], '29_0'], [['Abigail'], '30_0'], [['george'], '31_1'], [['Mark'], '32_0'], [['None'], '33'], [['Jhon'], '34_1'], [['Margaret'], '35_1'], [['Abigail', 'Margaret'], '36_0'], [['None'], '36_1'], [['Mark'], '37_0'], [[''], '37_1'], [['Mark']

# evaluating the 3 steps ( events extraction ,subject extraction , pronoun resolution )

In [468]:
#compar null_subject with subjects_ from labeled data
#compar null_event with events_ from labeled data

def evaluation(dialog_id=[],final_subjects_res_preprocessing=[],subjects_preprocessed=[]):

    count_true_negative=0
    count_true_positive=0
    count_false_positive=0
    count_false_negative=0
    a_list=set()
    b_list=set()
    c_list=set()
    d_list=set()
    o_list=set()

    count_other=0

    for i,j in enumerate(dialog_id):
        if str(dialog_id[i])==str(final_subjects_res_preprocessing[i][1]):
                for k,l in enumerate(subjects_preprocessed[i]):
                    for c,m in enumerate(final_subjects_res_preprocessing[i][0]):
                        if str(subjects_preprocessed[i][k]).lower().strip()==str(final_subjects_res_preprocessing[i][0][c]).lower().strip() and str(subjects_preprocessed[i][k]).lower().strip()=="none" and str(final_subjects_res_preprocessing[i][0][c]).lower().strip()=="none":
                            a_list.add(dialog_id[i])
                            count_true_negative+=1


    for i,j in enumerate(dialog_id):
        if dialog_id[i] not in a_list:
            if str(dialog_id[i])==str(final_subjects_res_preprocessing[i][1]):
                    for k,l in enumerate(subjects_preprocessed[i]):
                        for c,m in enumerate(final_subjects_res_preprocessing[i][0]):                        
                            if str(subjects_preprocessed[i][k]).lower().strip()==str(final_subjects_res_preprocessing[i][0][c]).lower().strip() and str(subjects_preprocessed[i][k]).lower().strip()!="none" and str(final_subjects_res_preprocessing[i][0][c]).lower().strip()!="none":
                                count_true_positive+=1
                                b_list.add(dialog_id[i])

    for i,j in enumerate(dialog_id):
        if dialog_id[i] not in a_list and dialog_id[i] not in b_list:
            if str(dialog_id[i])==str(final_subjects_res_preprocessing[i][1]):
                    for k,l in enumerate(subjects_preprocessed[i]):
                        for c,m in enumerate(final_subjects_res_preprocessing[i][0]):                    
                            if str(subjects_preprocessed[i][k]).lower().strip()!=str(final_subjects_res_preprocessing[i][0][c]).lower().strip() and str(subjects_preprocessed[i][k]).lower().strip()=="none" and str(final_subjects_res_preprocessing[i][0][c]).lower().strip()!="none":
                                count_false_positive+=1
                                c_list.add(dialog_id[i])

    for i,j in enumerate(dialog_id):
        if dialog_id[i] not in a_list and dialog_id[i] not in b_list and dialog_id[i] not in c_list:
            if str(dialog_id[i])==str(final_subjects_res_preprocessing[i][1]):
                    for k,l in enumerate(subjects_preprocessed[i]):
                        for c,m in enumerate(final_subjects_res_preprocessing[i][0]):                        
                            if str(subjects_preprocessed[i][k]).lower().strip()!=str(final_subjects_res_preprocessing[i][0][c]).lower().strip() and str(subjects_preprocessed[i][k]).lower().strip()!="none" and str(final_subjects_res_preprocessing[i][0][c]).lower().strip()=="none":
                                count_false_negative+=1
                                d_list.add(dialog_id[i])

    for i,j in enumerate(dialog_id):
        if dialog_id[i] not in a_list and dialog_id[i] not in b_list and dialog_id[i] not in c_list and dialog_id[i] not in d_list:
            if str(dialog_id[i])==str(final_subjects_res_preprocessing[i][1]):
                    for k,l in enumerate(subjects_preprocessed[i]):
                        for c,m in enumerate(final_subjects_res_preprocessing[i][0]):                            
                            if (str(subjects_preprocessed[i][k]).lower().strip()!=str(final_subjects_res_preprocessing[i][0][c]).lower().strip()) and (str(subjects_preprocessed[i][k]).lower().strip()!="none") and (str(final_subjects_res_preprocessing[i][0][c]).lower().strip()!="none"):
                                count_other+=1
                                o_list.add(dialog_id[i])
                    
    
    
    return a_list,b_list,c_list,d_list,o_list
    
#compare between subjects_ which is the labeled and preprocessed subject in data  .. with subject results after fill empty results 
#with Null value 


In [469]:
a_list,b_list,c_list,d_list,o_list=evaluation(dialog_id,final_subjects_res_preprocessing,subjects_preprocessed)

print("true_negative for subjects with pronoun resolution  : ",len(a_list))
print("true_positive for subjects with pronoun resolution  : ",len(b_list))
print("false_positive for subjects with pronoun resolution  : ",len(c_list)+len(o_list))
print("false_negative for subjects with pronoun resolution  : ",len(d_list))
    
#Recall = TruePositives / (TruePositives + FalseNegatives)
#Precision = TruePositives / (TruePositives + FalsePositives)
print("Precision ",(float)(len(b_list)/(float)((len(b_list))+(len(c_list)+len(o_list)))))
print("Recall ",(float)(len(b_list)/(float)(len(b_list)+len(d_list))))

true_negative for subjects with pronoun resolution  :  17
true_positive for subjects with pronoun resolution  :  127
false_positive for subjects with pronoun resolution  :  10
false_negative for subjects with pronoun resolution  :  4
Precision  0.927007299270073
Recall  0.9694656488549618


In [470]:
a_list,b_list,c_list,d_list,o_list=evaluation(dialog_id,final_subject_befor_resolution_res_preprocessing,subjects_no_resolution_preprocessed)

print("true_negative for subjects befor pronoun resolution  : ",len(a_list))
print("true_positive for subjects befor pronoun resolution  : ",len(b_list))
print("false_positive for subjects befor pronoun resolution  : ",len(c_list)+len(o_list))
print("false_negative for subjects befor pronoun resolution  : ",len(d_list))

#Recall = TruePositives / (TruePositives + FalseNegatives)
#Precision = TruePositives / (TruePositives + FalsePositives)
print("Precision ",(float)(len(b_list)/(float)((len(b_list))+(len(c_list)+len(o_list)))))
print("Recall ",(float)(len(b_list)/(float)(len(b_list)+len(d_list))))

true_negative for subjects befor pronoun resolution  :  17
true_positive for subjects befor pronoun resolution  :  129
false_positive for subjects befor pronoun resolution  :  8
false_negative for subjects befor pronoun resolution  :  4
Precision  0.9416058394160584
Recall  0.9699248120300752


In [471]:
a_list,b_list,c_list,d_list,o_list=evaluation(dialog_id,final_events_res_preprocessing,events_preprocessed)

print("true_negative for events  : ",len(a_list))
print("true_positive for events  : ",len(b_list))
print("false_positive for events  : ",len(c_list)+len(o_list))
print("false_negative for events  : ",len(d_list))
    
#Recall = TruePositives / (TruePositives + FalseNegatives)
#Precision = TruePositives / (TruePositives + FalsePositives)
print("Precision ",(float)(len(b_list)/(float)((len(b_list))+(len(c_list)+len(o_list)))))
print("Recall ",(float)(len(b_list)/(float)(len(b_list)+len(d_list))))

true_negative for events  :  17
true_positive for events  :  132
false_positive for events  :  5
false_negative for events  :  4
Precision  0.9635036496350365
Recall  0.9705882352941176


# some testing example on finding subjects functions 

In [109]:
print(find_sub_for_verb(nlp("my husband's parents flew out to meet my family when we got married in my hometown , so that was great "
),"married"))
print(find_sub_for_verb(nlp("i've got a really bad headache and my stomach hurts"),"hurts"))

print(find_sub_for_verb(nlp("people divorced every year"),"divorced"))
print(find_sub_for_verb(nlp("karla's sun have an exam"),"have"))
print(find_sub_for_verb(nlp("my brother have an exam"),"have"))

[we]
[(my, stomach)]
None
[(karla, sun)]
[(my, brother)]


In [110]:
print(find_sub_for_verb(nlp("sam and george have an exam"),"have"))

[sam, george]


In [111]:
 
print(find_sub_for_verb(nlp("i'm interviewed on montana public radio today about my life"),"interviewed"))

[i]


In [112]:
print(find_sub_for_verb(nlp("i heard she and george and sam has passed the bar exam and married recently "),"married"))

[she, george, sam]


In [113]:

print(find_sub_for_verb(nlp("george went for a job interview earlier and got the job "),"got"))

[george]


In [114]:
print(findSubs_for_actionEvents(nlp("george went for a job interview earlier and got the job"),verb_with_prepo))


[george]


In [115]:
print(findSubs_for_nounsEvents(nlp("george went for a job interview earlier and got the job"),nouns))

[george]


In [116]:
print(find_sub_for_verb(nlp("today my wife and i celebrate our 1st wedding anniversary "),"celebrate"))

[(my, wife), i]


In [117]:
print(find_sub_for_verb(nlp("i heard she has passed the bar exam and married recently "),"passed"))

[she]


In [118]:
print(find_sub_for_verb(nlp("joy's father died and he is very depressed"),"died"))

[(joy, father)]


In [119]:
print(find_sub_for_verb(nlp("Marla will have an surgery next week "),"have"))

[Marla]


In [120]:
print(find_sub_for_verb(nlp("nathan and i married last week "),"married"))

[nathan, i]


In [121]:
print(find_sub_for_verb(nlp("tomas divorced katty and take her alone ."),"divorced"))

[tomas]


In [122]:
print(find_sub_for_verb(nlp("you and me will married this month ."),"married"))

[you, me]


In [123]:
print(findSubs_for_nounsEvents(nlp("Marla will have an surgery next week "),nouns))

[Marla]


In [124]:
print(findSubs_for_verbEvents(nlp("i've got a really bad headache and my stomach and george hurts"),verbs_without_need_objects))


[(my, stomach), george]


In [125]:
print(findSubs_for_nounsEvents(nlp("(you are invited to my wedding"),nouns))


[my]


In [126]:
print(findSubs_for_verbEvents(nlp("i heard your son recently graduated"),verbs_without_need_objects))


[(your, son)]


In [127]:
print(findSubs_for_verbEvents(nlp("my husband flew out to meet my family when we got married in my hometown "),verbs_without_need_objects))


[(my, family)]


In [128]:
print(findSubs_for_verbEvents(nlp("defending my thesis was great"),verbs_with_objects))


[(my, thesis)]


In [129]:
print(findSubs_for_verbEvents(nlp("karla's sun passed the exam in 1999"),verbs_with_objects))


[(karla, sun)]


In [130]:
print(findSubs_for_verbEvents(nlp("do you want to go out to celebrate my good news"),verbs_with_objects))


[you]


In [131]:
print(findSubs_for_verbEvents(nlp("should probably let u all know that george and i are now engaged"),verbs_without_need_objects))


[george, i]


In [132]:
print(findSubs_for_verbEvents(nlp("my paper and george was published in 1999"),verbs_with_objects))


[(my, paper), george]


In [133]:
print(findSubs_for_verbEvents(nlp("the studio interviewed me last year"),verbs_without_need_objects))


[me]


In [134]:
print(findSubs_for_verbEvents(nlp("the studio interviewed my boss last year"),verbs_without_need_objects))


[(my, boss)]


In [135]:
print(findSubs_for_direct_relation(nlp("my wedding was last week"),noun_direct_relation))


[my]


In [136]:
print(findSubs_for_adjectiveEvents(nlp("i'm very sick and i have a fever"),adjectives))


[i]


In [137]:
print(findSubs_for_nounsEvents(nlp("george will have an surgery next week "),nouns))


[george]


In [138]:
print(findSubs_for_nounsEvents(nlp("kareem get to finish his masters "),nouns))


[his]


In [139]:
print(findSubs_for_nounsEvents(nlp("kareem get to finish masters "),nouns))

[kareem]


In [140]:
print(findSubs_for_actionEvents(nlp("we are moving to palmera last week"),verb_with_prepo))

[we]


In [141]:
print(findSubs_for_nounsEvents(nlp("kareem get to finish masters "),nouns))


[kareem]
